In [ ]:
## only do it the first time (by removing the #)
#import sys
#!{sys.executable} -m pip install plotly
#!{sys.executable} -m pip install cvxpy

In [18]:
import plotly.graph_objects as go 
import numpy as np
import cvxpy as cp
from time import time
from copy import copy, deepcopy
import pandas as pd
COLORS = {"school" : "green", "warehouse": "blue"}
TITLE = 'WFP Inventory problem'

# Defining framework 

Here we define the framework for visualization, then another function will solve the problem, and give us the decision variables $x_{mk}^t$ (how much food is delivered to school $m$ during tour $k$ at time $t$) and $y_k^t$ (boolean to know if the tour $k$ should be done at time $t$)

In [19]:
class School :
    def __init__(self, position, capacity,consumption,name):
        self.name = name
        self.pos = position
        self.C = capacity
        self.inventory = capacity
        self.q = consumption

    def eat(self):
        self.inventory -= self.q
        
    def receive(self,x):
        self.inventory += x
        
class Warehouse : 
    def __init__(self, position, capacity,name):
        self.name = name
        self.pos = position
        self.C = capacity
        self.inventory = capacity

    def deliver(self, quantity):
        self.inventory -= quantity

class Route : 
    def __init__(self, warehouse, SCHOOLS):
        self.w = warehouse
        self.S = SCHOOLS
        self.Mk = len(self.S)

    def set_Q(self,Q):
        self.Q = Q      

    def compute_edges(self):
        '''
        Make a list of edges for the route, that is, for each edge, a couple of objects Warehouse or School
        '''
        self.edges = [(self.w,self.S[0])]
        for i in range(self.Mk-1):
            self.edges.append((self.S[i],self.S[i+1]))
        self.edges.append((self.S[-1],self.w))

    def cost(edge,D):
        return D.loc[edge[0].name, edge[1].name]

    def compute_costs(self,D,fixed_costs=0.):
        '''
        compute the cost of a tour with respect to the distances and the fixed cost
        '''
        self.costs = []
        for e in self.edges :
            self.costs.append(Route.cost(e,D))
        
        self.cost = sum(self.costs) + fixed_costs

    def compute_C(self):
        self.room_available = [s.C-s.inventory for s in self.S]
        self.C = min(self.Q, sum(self.room_available))
                
    def do_tour(self):
        '''
        Change the inventory with respect to a tour and its X
        '''
        self.w.deliver(sum(self.X))
        for i in range(len(self.X)):
            self.S[i].receive(self.X[i])

    def make_arrows(self) : 
        self.arrows= []
        for e in range(self.Mk+1):
            edge = self.edges[e]
            x1, x2, y1, y2 = edge[0].pos[0], edge[1].pos[0], edge[0].pos[1], edge[1].pos[1]
            text = "cost = "+ str(self.costs[e])

            arrow = go.Scatter(x=[(7*x1+x2)/8,(x1+7*x2)/8],
                    y=[(7*y1+y2)/8,(y1+7*y2)/8],
                    line= dict(color="red"),
                    text = [text,text],
                    hoverinfo='text',
                    visible = False
                    )

            self.arrows.append(arrow)
            
    def set_no(self,indices):
        names = [s.name for s in self.S]
        self.numbers_RtoS = np.array([indices[name] for name in names  ])


In [20]:
class Map :
    def __init__(self, schools, warehouses, possible_routes = [], Q = 5.):
        '''
        This object represents the state of the problem at some time t 
        '''
        self.S = schools
        self.W = warehouses
        self.R_possible = possible_routes
        self.Q = Q
        self.M, self.N, self.K = len(schools),len(warehouses), len(possible_routes )
        find_index_s =  {schools[i].name : i for i in range(self.M)}
        for r in self.R_possible : 
            r.set_Q(Q)
            r.set_no(find_index_s)

        
        self.t = 0.
        self.total_cost = 0
        self.cost = 0
        
        for k in range(self.K): 
            self.R_possible[k].number = k

    def build_Rmat(self):
        '''
        Method that build the numpy array Rmat such that r[k,m] = 1 if & only if S[m] in R[k]
        '''
        self.Rmat = np.zeros((self.K,self.M),dtype = bool)
        names = [s.name for s in self.S]
        for k in range(self.K):
            r = self.R_possible[k]
            for stop in r.S : 
                m = names.index(stop.name)
                self.Rmat[k,m] = True

    def compute_edges(self):
        for r in self.R_possible : 
            r.compute_edges()

    def compute_costs(self,D, fixed_costs=0.):
        for r in self.R_possible : 
            r.compute_costs(D, fixed_costs=fixed_costs)

    def select_tours(self,y):
        # y should be a boolean vector of length K here that state if the tour k should be done
        self.R = np.arange(self.K)[y]
        
    def compute_X(self,x):
        # x is a matrix MxK that gives the quantity of food to be delivered to each school for each tour
        for k in self.R : 
            r = self.R_possible[k]
            r.X = x[k,r.numbers_RtoS]
                        
    def do_tours(self):
        for k in self.R :
            r = self.R_possible[k] 
            r.do_tour()
            self.cost += r.cost

        self.total_cost += self.cost
        self.title = TITLE + "        Cost = %s          Total Cost = " %str(self.cost) + str(self.total_cost)
        self.cost = 0

    def eat(self):
        for s in self.S:
            s.eat()

    def init_draw(self):

        # create arrows
        for k in range(self.K): 
            r = self.R_possible[k].make_arrows()

        #plot the schools
        plot_schools = go.Scatter(x=[school.pos[0] for school in self.S],
                        y=[school.pos[1] for school in self.S],
                        mode='markers',
                        name='schools',
                        marker=dict(symbol='circle-dot',
                                        size=50,
                                        color=COLORS["school"]
                                        ),
                        text=[s.name+" C = "+str(s.C)+"  ;   q = "+str(s.q) for s in self.S],
                        hoverinfo='text',
                        opacity=0.8
                        )
        #plot the warehouses
        plot_warehouses = go.Scatter(x=[warehouse.pos[0] for warehouse in self.W],
                            y=[warehouse.pos[1] for warehouse in self.W],
                            mode='markers',
                            name='warehouses',
                            marker=dict(symbol='circle-dot',
                                            size=70,
                                            color=COLORS["warehouse"]
                                            ),
                            text=[w.name+" C = "+str(w.C) for w in self.W],
                            hoverinfo='text',
                            opacity=0.8
                            )

        axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=True,
            showgrid=True,
            showticklabels=True,
            )

        self.title = TITLE + "        Cost = %s          Total Cost = " %str(self.cost) + str(self.total_cost)

        couples = [(school.pos, "I = "+str(school.inventory),'black') for school in self.S] + [(warehouse.pos, "I = "+str(warehouse.inventory),'yellow') for warehouse in self.W]
        layout = dict(
            title= self.title,
              annotations= Map.make_annotations(couples), 
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)',
              updatemenus= []
        )

        self.layout = layout
        self.data = [plot_schools,plot_warehouses]
        self.arrows = [-1,-1]
        for r in self.R_possible : 
            number = r.number
            for x in r.arrows : 
                self.data.append(x)
                self.arrows.append(number)
       
    def make_annotations(couples):
        annotations = []
        for (pos,txt,color) in couples:
            annotations.append(
                dict(
                    text=txt, # text within the circles
                    x=pos[0], y=pos[1],
                    xref='x1', yref='y1',
                    font=dict(color=color, size=15),
                    showarrow=False)
            )
        return annotations

    def make_updatemenu(self):
        couples = [(school.pos, "I = "+str(round(school.inventory,2)),'black') for school in self.S] + [(warehouse.pos, "I = "+str(warehouse.inventory),'yellow') for warehouse in self.W]
        l = len(self.data)
        visible = [True]*2 + [False]*(l-2)

        if self.t.is_integer():
            period = " (before lunch)"
            for k in self.R : 
                r = self.R_possible[k]
                for i in range(r.Mk):
                    e = r.edges[i]
                    x = (e[0].pos[0]+e[1].pos[0]) / 2 + .5
                    y = (e[0].pos[1]+e[1].pos[1]) / 2 + .5
                    couples.append(([x,y], str(round(r.X[i],2))+ "  ",'red'))

                i = self.arrows.index(r.number)
                a = len(r.arrows)
                visible[i:i+a]=[True]*a
        
        else : 
            period = "  (evening)"
            visible[2:] = [False]*(l-2)


        annotations = Map.make_annotations(couples)

        return dict(label="t = "+str(int(self.t))+ period, method = "update", args=[{"visible" : copy(visible)  },{"annotations": annotations, "title":self.title }])

    def run(self,D,solver,T=10, H=4):
        '''
        final function that make the process continue for T days, and plot it into self.fig 
        Also have to put as an input a function f that will build X and Y
        '''
        self.build_Rmat()
        self.compute_edges()
        self.compute_costs(D)
        
        t0 = time()
        X,Y = solver(cost=np.array([r.cost for r in self.R_possible]),
                q      =np.array([s.q for s in self.S]),
                C      =np.array([s.C for s in self.S]),
                I_init = np.array([s.inventory for s in self.S]),
                r      =self.Rmat,
                Q=self.Q,T=T,H=H)
        
        print("Running time for solver is %f sec" %(time()-t0))
        self.init_draw()
        L1,L2 = [],[]
        for i in range(T):
            #morning
            self.select_tours(Y[i])
            self.compute_X(X[i])
            self.do_tours()
            L1.append(self.make_updatemenu())
            self.R = []
            self.t += 0.5
            
            # evening
            self.eat()
            L2.append(self.make_updatemenu())
            self.t += 0.5



        self.layout["updatemenus"]    = [ dict(buttons = L1, direction = "up",x=0.,y=0.),dict(buttons = L2, direction = "up",x=0.3,y=0.) ]
        

        self.fig = dict(data=self.data, layout=self.layout)


# Defining solver

\begin{eqnarray}
&\min& \sum_t \sum_k c_k y_k^t \\
& & \sum_m x_{km}^t \leq Q y_k^t \qquad \forall t, k \\
& & x_{km}^t \leq Q r_{km} \qquad \forall t,m, k\\
& & q_m  \leq  I_m^0 + \sum_{k} \sum_{s\leq t}  x_{km}^s - t q_m \leq C_m \qquad \forall t,m \\
\end{eqnarray}

In [21]:
def solver1(cost,q,C,r,I_init,Q,T,H=None):
    M,K = len(q),len(cost)
    
    Y = cp.Variable((T,K),boolean=True)
    
    X = {}
    constraints = []
    dI = 0
    for t in range(T):
        X[t] = cp.Variable((K,M),nonneg=True)
        
        constraints.append(cp.sum(X[t],axis=1)<=Q*Y[t] )
        constraints.append(X[t]<=Q*r)
        
        dI = dI + cp.sum(X[t],axis=0) 
        constraints.append( dI <= C+t*q-I_init )
        constraints.append( dI >= (t+1)*q-I_init )
        
    
    M = np.array([cost for t in range(T)]).T
    problem = cp.Problem(cp.Minimize(cp.trace(M @ Y)), constraints)
    problem.solve()
    
    return np.array([X[t].value for t in range(T)]),np.array(Y.value,dtype=bool)

In [22]:
def solver2(cost,q,C,r,I_init,Q,T,H=None):
    if H is None : 
        return solver1(cost,q,C,r,I_init,Q,T,H=None)
    
    else : 
        X_weekly, Y_weekly = [],[]
        t = 0
        I = I_init[:]
        while t<T : 
            X,Y = solver1(cost,q,C,r,I,Q,H)
            X_weekly.append(X),Y_weekly.append(Y)
            t = t + H
            I = I - H*q + X.sum(axis=0).sum(axis=0)
            
        return np.concatenate(X_weekly,axis=0), np.concatenate(Y_weekly,axis=0)
            
            
            
            

# Example 1 

## Set data

In [23]:
w1 = Warehouse(position=[0.,0.],capacity = 100, name='w1')

positions = [
    [0.,10.],
    [3.,10.],
    [10.,3.],
    [10.,0.]
]
capacities   = [5.,3.,2.,4.]
consumptions = [1.,3.,2.,1.5]
tours = [
    [0],[1],[2],[3],[0,1],[1,2],[2,3]
] 
# distance matrix
d = [
    [  0,100,100,100,100],
    [100,  0, 10,150,160],
    [100, 10,  0,140,150],
    [100,150,140,  0, 10],
    [100,160,150, 10,  0]
]

In [24]:
schools = []
for i in range(len(positions)):
    schools.append(  School(positions[i], capacities[i], consumptions[i] , 's%i'%i )  )

routes = [ Route(w1, [schools[m] for m in tour] )  for tour in tours ]



# transform it to a dataframe
names = [w1.name]+[s.name for s in schools]
D = pd.DataFrame(data=d, columns=names, index=names)

MAP1 = Map(
    schools=schools,
    warehouses=[w1],
    possible_routes=routes,
    Q = 5.
    )

## Run

solve the problem during 30 days, by computing every 5days the optimal solution for the next 5 days (H=5 is kind of the maximum here)

In [25]:
MAP1.run(D,solver2,T=30,H=5)
go.Figure(MAP1.fig).show()

Running time for solver is 2.020858 sec
